In [0]:
# 1. Tell Git about yourself:
from getpass import getpass
u = input('Username: ') # Git username
p = getpass('Password: ') # Git password

Username: mdmeldon
Password: ··········


In [0]:
# 2. Initialize Git, then Pull the Branch:
! git init
! git remote add origin https://$u:$p@github.com/mdmeldon/DS-CV.git 

Initialized empty Git repository in /content/.git/


In [18]:
! git pull origin master

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/mdmeldon/DS-CV
 * branch            master     -> FETCH_HEAD
   ef480dc..e0c073e  master     -> origin/master
Updating ef480dc..e0c073e
Fast-forward
 colab.sh | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [19]:
# 3. Run helper:
! sh colab.sh

rm: cannot remove 'sample_data': No such file or directory


In [20]:
! dvc pull

+-------------------------------------------+
|                                           |
|     Update available 0.93.0 -> 0.94.0     |
|      Run `pip install dvc --upgrade`      |
|                                           |
+-------------------------------------------+

!

!



!





!




                                                                                

                                                                                


                                                                                



5 added


In [0]:
# sklearn
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#import torchvision
from resnet import resnet50
from auc import AUCCallback
import torch
from torch.utils.data import DataLoader, TensorDataset
from catalyst.dl import SupervisedRunner
from catalyst.dl.utils import set_global_seed, prepare_cudnn
from catalyst.dl.callbacks import AccuracyCallback, PrecisionRecallF1ScoreCallback, VerboseLogger

# Other  
from tqdm import tqdm, tqdm_pandas
import scipy
from scipy.stats import skew
import librosa
import librosa.display
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import pandas as pd
import seaborn as sns
import glob 
import os
import sys
import IPython.display as ipd  # To play sound in the notebook
import warnings
# ignore warnings 
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [27]:
ref = pd.read_csv("data/processed/Data_path.csv")
ref.head()

,labels,source,path
0,male_disgust,SAVEE,../data/raw/surrey-audiovisual-expressed-emoti...
1,male_disgust,SAVEE,../data/raw/surrey-audiovisual-expressed-emoti...
2,male_sad,SAVEE,../data/raw/surrey-audiovisual-expressed-emoti...
3,male_neutral,SAVEE,../data/raw/surrey-audiovisual-expressed-emoti...
4,male_fear,SAVEE,../data/raw/surrey-audiovisual-expressed-emoti...


In [0]:
'''
1. Data Augmentation method   
'''
def speedNpitch(data):
    """
    Speed and Pitch Tuning.
    """
    # you can change low and high here
    length_change = np.random.uniform(low=0.8, high = 1)
    speed_fac = 1.2  / length_change # try changing 1.0 to 2.0 ... =D
    tmp = np.interp(np.arange(0,len(data),speed_fac),np.arange(0,len(data)),data)
    minlen = min(data.shape[0], tmp.shape[0])
    data *= 0
    data[0:minlen] = tmp[0:minlen]
    return data

'''
2. Extracting the MFCC feature as an image (Matrix format).  
'''
def prepare_data(df, n, aug, mfcc):
    X = np.empty(shape=(df.shape[0], n, 216, 1))
    input_length = sampling_rate * audio_duration
    
    cnt = 0
    for fname in tqdm(df.path):
        file_path = fname
        data, _ = librosa.load(file_path, sr=sampling_rate
                               ,res_type="kaiser_fast"
                               ,duration=2.5
                               ,offset=0.5
                              )

        # Random offset / Padding
        if len(data) > input_length:
            max_offset = len(data) - input_length
            offset = np.random.randint(max_offset)
            data = data[offset:(input_length+offset)]
        else:
            if input_length > len(data):
                max_offset = input_length - len(data)
                offset = np.random.randint(max_offset)
            else:
                offset = 0
            data = np.pad(data, (offset, int(input_length) - len(data) - offset), "constant")

        # Augmentation? 
        if aug == 1:
            data = speedNpitch(data)
        
        # which feature?
        if mfcc == 1:
            # MFCC extraction 
            MFCC = librosa.feature.mfcc(data, sr=sampling_rate, n_mfcc=n_mfcc)
            MFCC = np.expand_dims(MFCC, axis=-1)
            X[cnt,] = MFCC
            
        else:
            # Log-melspectogram
            melspec = librosa.feature.melspectrogram(data, n_mels = n_melspec)   
            logspec = librosa.amplitude_to_db(melspec)
            logspec = np.expand_dims(logspec, axis=-1)
            X[cnt,] = logspec
            
        cnt += 1
    
    return X


In [0]:
# sampling_rate=44100
# audio_duration=2.5
# n_mfcc = 30
# mfcc = prepare_data(ref, n = n_mfcc, aug = 0, mfcc = 1)

In [0]:
mfcc = np.load('data/processed/MFCC_PREPARE.npy')

In [0]:
# Split between train and test 
X_train, X_test, y_train, y_test = train_test_split(mfcc
                                                    , ref.labels
                                                    , test_size=0.25
                                                    , shuffle=True
                                                    , random_state=42
                                                   )


In [0]:
#np.save('../data/processed/MFCC_PREPARE.npy', mfcc)

In [0]:
# one hot encode the target 
lb = LabelEncoder()
# y_train = np.eye(len(ref.labels.unique()), dtype='uint8')[lb.fit_transform(y_train)] # to_categorical
# y_test = np.eye(len(ref.labels.unique()), dtype='uint8')[lb.fit_transform(y_test)] # to_categorical

y_train = lb.fit_transform(y_train) # to_categorical
y_test = lb.fit_transform(y_test) # to_categorical

# Normalization as per the standard NN process
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

In [0]:
model = resnet50(pretrained=False, progress=True, num_classes=14)

In [0]:
BATCH_SIZE = 16

In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# create feature and targets tensor for train set. As you remember we need variable to accumulate gradients. Therefore first we create tensor, then we will create variable
featuresTrain = torch.from_numpy(X_train)
targetsTrain = torch.from_numpy(y_train).type(torch.LongTensor) # data type is long

# create feature and targets tensor for test set.
featuresTest = torch.from_numpy(X_test)
targetsTest = torch.from_numpy(y_test).type(torch.LongTensor) # data type is long

trainSet = TensorDataset(featuresTrain,targetsTrain)
validSet = TensorDataset(featuresTest,targetsTest)

# data loader
train_loader = DataLoader(trainSet, batch_size = BATCH_SIZE, shuffle = True)
valid_loader = DataLoader(validSet, batch_size = BATCH_SIZE, shuffle = False)

In [0]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
criterion = torch.nn.CrossEntropyLoss()

In [0]:
# Init catalyst components
runner = SupervisedRunner(device=device)
loaders = {'train': train_loader, 'valid': valid_loader}
logdir = 'data/logs/0'
callbacks = [
             AUCCallback(),
             AccuracyCallback(),
    #AUCCallback(num_classes = 14),
    # VerboseLogger(),
    # PrecisionRecallF1ScoreCallback(num_classes = 14),
]

In [51]:
runner.train(
        model=model.double(),
        criterion=criterion,
        optimizer=optimizer,
        loaders=loaders,
        callbacks=callbacks,
        logdir=logdir,
        num_epochs=20,
        verbose=True,        
    )  







1/20 * Epoch (train):   0% 0/571 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/catalyst/core/state.py:407: DeprecationWarning:

`output` was deprecated, please use `batch_out` instead

/usr/local/lib/python3.6/dist-packages/catalyst/core/state.py:393: DeprecationWarning:

`input` was deprecated, please use `batch_in` instead



Streaming output truncated to the last 5000 lines.

1/20 * Epoch (train):  27% 153/571 [01:30<04:04,  1.71it/s, accuracy01=0.375, loss=2.230]





1/20 * Epoch (train):  27% 154/571 [01:30<04:03,  1.71it/s, accuracy01=0.375, loss=2.230]





1/20 * Epoch (train):  27% 154/571 [01:30<04:03,  1.71it/s, accuracy01=0.312, loss=2.235]





1/20 * Epoch (train):  27% 155/571 [01:30<04:03,  1.71it/s, accuracy01=0.312, loss=2.235]





1/20 * Epoch (train):  27% 155/571 [01:31<04:03,  1.71it/s, accuracy01=0.312, loss=2.059]





1/20 * Epoch (train):  27% 156/571 [01:31<04:02,  1.71it/s, accuracy01=0.312, loss=2.059]





1/20 * Epoch (train):  27% 156/571 [01:31<04:02,  1.71it/s, accuracy01=0.375, loss=2.053]





1/20 * Epoch (train):  27% 157/571 [01:31<04:01,  1.71it/s, accuracy01=0.375, loss=2.053]





1/20 * Epoch (train):  27% 157/571 [01:32<04:01,  1.71it/s, accuracy01=0.375, loss=1.947]





1/20 * Epoch (train):  28% 158/571 [01:32<04:01,  1.71it/s, accuracy01=0.375, loss=1.947]





AssertionError: ignored